In [ ]:
!conda create -n macsima python=3.10
!conda activate macsima
!pip install -r requiremnts.txt

In [ ]:
# read in json file (macsima output)
# get important info into format that makes sense
# save to excel and/or report


- overall exp metadata
-name
- run time - start time - end time

- blocks
- cycles
- magnification
- colour names
- detection channel

- background pictures without any staining
- another can just the region of interest
- time coefficient for each cycle

In [10]:
import json
import pandas as pd

In [ ]:
def load_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data

In [6]:
macsima_output_file = 'data/250128_macsima_output.json'
data = load_json(macsima_output_file)

In [92]:
data.keys()

dict_keys(['experiments', 'procedures', 'racks', 'reagents', 'rois', 'samples'])

In [ ]:
data['racks'][0].keys()
# slides
# 8 rois spread accros 4 racks
# which ROIs are from which rack

dict_keys(['analysisAlgorithmId', 'comment', 'contentType', 'creationDateTime', 'creator', 'id', 'isArchived', 'isTemplate', 'lotNumber', 'modificationDateTime', 'modifier', 'name', 'orderNumber', 'owner', 'preparationDateTime', 'qualityControlAlgorithmId', 'rackInfo', 'sampleGroupIds', 'sampleIds', 'scanDateTime', 'serialNumber', 'state', 'wells'])

In [32]:
data['experiments'][0]['name']

'250128_liver_OCT_FCRB 1'

In [ ]:
the sample info is associted with the rack_name
    and the 

within the ROI info we have a sample id

for each roi we want to know whuch spample its associated with

In [100]:
data['rois'][1]

{'autoFocus': {'exposureTime': 50,
  'method': 'AutofocusMethod_ConstantZ',
  'zPosition': 0},
 'comment': '',
 'creationDateTime': '',
 'creator': '',
 'description': '',
 'experimentId': '{9a237cfc-3501-4c97-a7d3-501016171e36}',
 'id': '{63fa342c-b633-43f8-846b-fea70d854b13}',
 'imageFileIds': ['{286acdf7-44ea-47e1-b577-4332c9160f97}',
  '{01c90bd4-32b3-4a97-b1b6-47d7c27d7957}',
  '{76da04c9-7a40-4a35-9e41-3fb836690165}',
  '{1e421b0e-c434-4a2e-a6c0-593e8a1928a4}',
  '{ef00d819-8224-4aa6-a808-42d8ce688d9d}',
  '{009b3ce4-25d7-47c9-9f51-27d9cf0d83c3}',
  '{68b895c4-b7ee-434f-ad1c-6246770a40ee}',
  '{2946efcf-c260-447d-a63a-c828716e33a9}',
  '{d7e8cc94-ab17-41b0-aeaa-a6d932b5f0da}',
  '{a6521f9e-0200-480a-9819-31e35ae034b1}',
  '{064c6b0f-eaf3-4245-bf35-a157ddc81877}',
  '{da714fd1-f646-4575-8912-a84162e2e73c}',
  '{c45258fc-e029-4ea6-827b-e99100125e75}',
  '{5087994c-8256-4054-ab42-af005541d908}',
  '{f49215e1-541b-44c2-ad7d-dcc8296aac58}',
  '{75e6ab86-4635-405c-a12f-ff3d1fc241e2}',


In [ ]:
experiment_name = data['experiments'][0]['name']
# 250128_liver_OCT_FCRB 1

In [ ]:
procedure_name = data['procedures'][0]['comment']
# standard procedure

In [79]:
data

{'experiments': [{'actualRunningTime': 53367,
   'averageROIPerWell': 1,
   'averageSizePerROI': 1,
   'comment': '',
   'conclusion': '',
   'creationDateTime': '2025-01-28T15:50:01Z',
   'creator': '{23a8d883-dba3-42b1-8ec3-343c5f4137e0}',
   'datasetSize': -1,
   'estimatedRunningTime': 50983,
   'executionEndDateTime': '2025-01-29T06:43:08Z',
   'executionStartDateTime': '2025-01-28T15:53:36Z',
   'experimenter': 'admin',
   'id': '{9a237cfc-3501-4c97-a7d3-501016171e36}',
   'instrumentId': '{d2ccd42e-8e22-4da1-b82b-37a58b004efa}',
   'isArchived': False,
   'isBackedUp': False,
   'isBioQC': False,
   'isExpress': False,
   'keyword': '',
   'modificationDateTime': '2025-01-29T06:43:10Z',
   'modifier': '{23a8d883-dba3-42b1-8ec3-343c5f4137e0}',
   'name': '250128_liver_OCT_FCRB 1',
   'organization': '',
   'owner': '',
   'parameters': '',
   'plannedExecutionDateTime': '',
   'project': '',
   'purpose': '',
   'runExperimentPaths': ['/raid/shared/experiments/250128_liver_OCT_FC

In [ ]:
# I have only run with one "rack" so far, 
# but the expt we have running now has 2, 
# so I will see if the file has diff info
#  or diff layout that would be important
rack_name = data['racks'][0]['name']
# '250128_FCRB'

In [61]:
#  'executionEndDateTime': '2025-01-29T06:43:08Z',
# 'executionStartDateTime': '2025-01-28T15:53:36Z',
start_time = data['experiments'][0]['executionStartDateTime']
end_time = data['experiments'][0]['executionEndDateTime']

# 'actualRunningTime':  53367,
actual_running_time_total_seconds = data['experiments'][0]['actualRunningTime']



In [ ]:
# Convert seconds to hours minutes and seconds from "actualRunningTime" which reports running time in seconds
def convert_seconds_to_hms(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds_left = seconds % 60
    return hours, minutes, seconds_left

# 14 hrs 49 min 27 sec

In [63]:
runtime_hours, runtime_minutes, runtime_seconds = convert_seconds_to_hms(actual_running_time_total_seconds)

In [64]:
runtime_hours, runtime_minutes, runtime_seconds

(14, 49, 27)

In [65]:
# usedDiskspace (I don't know what unit this is,
#  but I can check and then it would be good to report in GB)
# 'usedDiskspace': 178364,
used_disk_space = data['experiments'][0]['usedDiskspace']

In [ ]:
# I need to check the units; 
# also ROIS listed after reagents in json; 
# also note we usually have one ROI but we could have more
rois_shape_data = data['rois'][0]['shape']
rois_shape_type = data['rois'][0]['type']


{'Data': '{"Height":10,"Width":19}',
 'Type': 'ShapeType_Rectangle',
 'XPosition': 0,
 'YPosition': 0}

In [73]:
sample_name = data['samples'][0]['name']
sample_species = data['samples'][0]['species']
sample_type = data['samples'][0]['sampleType']
sample_organ = data['samples'][0]['organ']
fixation_method = data['samples'][0]['fixationMethod']

In [82]:
len(data['rois'])

2

In [86]:
autofocus_methods = [data['rois'][i]['autoFocus']['method'] for i in range(len(data['rois']))]

In [90]:
blocks = data['procedures'][0]['blocks']

In [ ]:
len(blocks)

18

In [ ]:
separate column for height width shapre
    
    height and widt h10 and 19 whre did that come frm and we dont need it

what does ROI stand for